In [ ]:
from google.colab import files
files.upload()

In [ ]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [ ]:
from bs4 import BeautifulSoup as soup
import urllib
import re
from typing_extensions import Text
from html.parser import HTMLParser

# 1. Compare the data of Semrush and Webometrics.

In [ ]:
df = pd.read_csv("Compare the number of backlinks at Webometrics and Semrush.csv")
df

,STT,Trường đại học,Thứ tự ở Webometrics,Thứ tự tại Semrush
0,1,Harvard University,1,1
1,2,Massachusetts Institute of Technology,2,3
2,3,Stanford University,3,2
3,4,University of California Berkeley,4,4
4,5,Cornell University,5,9
5,6,University of Washington,6,10
6,7,University of Michigan,7,11
7,8,Columbia University New York,8,5
8,9,Yale University,9,16
9,10,Pennsylvania State University,10,8


In [ ]:
#Semrush
semrush_backlink = df['Thứ tự tại Semrush']
#Webometrics
webometrics_backlink = df['Thứ tự ở Webometrics']

print("Semrush:\n", semrush_backlink)
print("Webometrics:\n", webometrics_backlink)

# compute cosine similarity
cosine_backlink = np.dot(semrush_backlink,webometrics_backlink)/(norm(semrush_backlink)*norm(webometrics_backlink))

print("Cosine Similarity vs Webometrics:", cosine_backlink)

Semrush:
 0      1
1      3
2      2
3      4
4      9
5     10
6     11
7      5
8     16
9      8
10     6
11    12
12    13
13    14
14    17
15    15
16     7
17    20
18    18
19    19
Name: Thứ tự tại Semrush, dtype: int64
Webometrics:
 0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
Name: Thứ tự ở Webometrics, dtype: int64
Cosine Similarity vs Webometrics: 0.9567944250871079


# 2. Collect the number of citations on Google Scholar.


In [ ]:
df1 = pd.read_csv("University Website.csv")
df1

,ID,University,University Website
0,1,Harvard University,https://www.harvard.edu/
1,2,Stanford University,https://www.stanford.edu/
2,3,Massachusetts Institute of Technology,https://www.mit.edu/
3,4,University of California Berkeley,https://www.berkeley.edu/
4,5,University of Oxford,https://www.ox.ac.uk/
5,6,University of Michigan,https://umich.edu/
6,7,University of Washington,https://www.washington.edu/
7,8,Cornell University,https://www.cornell.edu/
8,9,Columbia University New York,https://www.columbia.edu/
9,10,Johns Hopkins University,https://www.jhu.edu/


In [ ]:
#Information about the file
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  10 non-null     int64 
 1   University          10 non-null     object
 2   University Website  10 non-null     object
dtypes: int64(1), object(2)
memory usage: 368.0+ bytes


In [ ]:
df2 = pd.read_csv("Criteria Score.csv")
df2

,ID,University,Visibility Score,Openness Score,Excelence Score
0,1,Harvard University,3400000000,50658768,951
1,2,Stanford University,774500000,40442825,451
2,3,Massachusetts Institute of Technology,279300000,26265138,845
3,4,University of California Berkeley,270200000,26077114,256
4,5,University of Oxford,101600000,25959095,165
5,6,University of Michigan,108000000,23762098,173
6,7,University of Washington,127900000,6864628,133
7,8,Cornell University,549000000,18550524,144
8,9,Columbia University New York,166300000,23110488,120
9,10,Johns Hopkins University,64500000,12017656,96


In [ ]:
#Information about the file
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                10 non-null     int64 
 1   University        10 non-null     object
 2   Visibility Score  10 non-null     int64 
 3   Openness Score    10 non-null     int64 
 4   Excelence Score   10 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 528.0+ bytes


In [ ]:
#statistics of the links
def get_link(text_link):
  s = soup(str(urllib.request.urlopen(text_link).read()), 'lxml')
  next_page =list(map(Text, s.find_all('button', {'class':'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx'})))
  # print(next_page)
  onclick_text = ''
  if(next_page[0].find('onclick') == -1):
    return 'x'
  else:
    onclick_text = (next_page[0].split(' ')[8]).split("/")[1][:-3]
    # onclick_text = convert(onclick_text)
    onclick_text = onclick_text.replace('\\\\x3d','=')
    onclick_text = onclick_text.replace('\\\\x26','&')
    onclick_text = onclick_text.replace('&oe=ASCII','')
    return onclick_text
  # print(next_page)


In [ ]:
#function to count the number of citations from one link
def count_cititation(citiation_link):
  s = soup(str(urllib.request.urlopen(citiation_link).read()), 'lxml')
  citation_text = map(Text, [i.text for i in s.find_all('div', {'class':'gs_ai_cby'})])
  citation_text = list(citation_text)
  citation_number = []
  for num in citation_text:
    citation_number += re.findall(r'\d+', num)
  total = 0
  for num in citation_number:
    total = total + int(num)

  #print(citation_number)
  #print(total)
  return total

In [ ]:
# Function to extract domain
def get_domain(url):
  start = url.find("://") + 3
  end = url.find("/", start)
  domain = url[start:end]
  if "www." in domain:
      domain = domain[4:]
  return domain


In [ ]:
ggsholar_link = []
for i in range(len(df1)):
  gglink = 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=' + get_domain(df1['University Website'][i]) + '&btnG='
  ggsholar_link.append(gglink)
ggsholar_link

['https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=harvard.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=stanford.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=mit.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=berkeley.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=ox.ac.uk&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=umich.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=washington.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=cornell.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=columbia.edu&btnG=',
 'https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=jhu.edu&btnG=']

In [ ]:
#Count the total number of citations
for i in range(len(df1)):
  text_link = ggsholar_link[i]
  check = 1
  n = 29
  citiation_total = 0
  while(check != 0):
    #retrieve all links for statistics
    onclick_text = get_link(text_link)
    if(onclick_text == 'x'):
      check = 0
    else:
      text_link = 'https://scholar.google.com/'+ onclick_text
      # Count the total number of citations
      citiation_total += count_cititation(text_link)
    n -= 1
    if(n < 0):
      break
  df2['Openness Score'][i] = citiation_total


<ipython-input-14-54ec6d206849>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Openness Score'][i] = citiation_total


In [ ]:
#The result of Openness Score after the statistics
df2

,ID,University,Visibility Score,Openness Score,Excelence Score
0,1,Harvard University,3400000000,28141176,951
1,2,Stanford University,774500000,23006786,451
2,3,Massachusetts Institute of Technology,279300000,15603655,845
3,4,University of California Berkeley,270200000,15700111,256
4,5,University of Oxford,101600000,14384636,165
5,6,University of Michigan,108000000,12536070,173
6,7,University of Washington,127900000,5032077,133
7,8,Cornell University,549000000,10471284,144
8,9,Columbia University New York,166300000,13409770,120
9,10,Johns Hopkins University,64500000,7279267,96


# 3. Rank according to three criteria

In [ ]:
new_col_rank = []
max_value_VisibilityScore = float(df2['Visibility Score'].max())
max_value_OpennessScore = float(df2['Openness Score'].max())
max_value_ExcelenceScore = float(df2['Excelence Score'].max())

for i in range(len(df2)):
    # Perform the necessary calculations and save the results to new_col_data
    result = (float(df2['Visibility Score'][i] * 0.5))/max_value_VisibilityScore + (float(df2['Openness Score'][i] * 0.1))/max_value_OpennessScore + (float(df2['Excelence Score'][i] * 0.4))/max_value_ExcelenceScore
    new_col_rank.append(result)


# Add a new column with data
df2['Rank Score'] = new_col_rank

# Save data to a CSV file
df2.to_csv('data_rank.csv', index=False)
df2

,ID,University,Visibility Score,Openness Score,Excelence Score,Rank Score
0,1,Harvard University,3400000000,28141176,951,1.000000
1,2,Stanford University,774500000,23006786,451,0.385347
2,3,Massachusetts Institute of Technology,279300000,15603655,845,0.451937
3,4,University of California Berkeley,270200000,15700111,256,0.203202
4,5,University of Oxford,101600000,14384636,165,0.135458
5,6,University of Michigan,108000000,12536070,173,0.133195
6,7,University of Washington,127900000,5032077,133,0.092631
7,8,Cornell University,549000000,10471284,144,0.178513
8,9,Columbia University New York,166300000,13409770,120,0.122581
9,10,Johns Hopkins University,64500000,7279267,96,0.075731


In [ ]:
# The fillna(0) method is used to replace NaN values in the 'RANK' column with 0
df2['Rank Score'] = df2['Rank Score'].fillna(0)
df2['Visibility Score'] = df2['Visibility Score'].fillna(0)
df2['Openness Score'] = df2['Openness Score'].fillna(0)
df2['Excelence Score'] = df2['Excelence Score'].fillna(0)
# The parameter na_option='keep' is used to keep NaN values in the 'RANK' column after calculating the rank
df2['University Rank'] = (df2['Rank Score'].rank(ascending=False, na_option='keep'))
df2['Visibility Rank'] = (df2['Visibility Score'].rank(ascending=False, na_option='keep'))
df2['Openness Rank'] = (df2['Openness Score'].rank(ascending=False, na_option='keep'))
df2['Excelence Rank'] = (df2['Excelence Score'].rank(ascending=False, na_option='keep'))
# Save data to a CSV file
df2.to_csv('data_rank.csv', index=False)
df2

,ID,University,Visibility Score,Openness Score,Excelence Score,Rank Score,University Rank,Visibility Rank,Openness Rank,Excelence Rank
0,1,Harvard University,3400000000,28141176,951,1.000000,1.0,1.0,1.0,1.0
1,2,Stanford University,774500000,23006786,451,0.385347,3.0,2.0,2.0,3.0
2,3,Massachusetts Institute of Technology,279300000,15603655,845,0.451937,2.0,4.0,4.0,2.0
3,4,University of California Berkeley,270200000,15700111,256,0.203202,4.0,5.0,3.0,4.0
4,5,University of Oxford,101600000,14384636,165,0.135458,6.0,9.0,5.0,6.0
5,6,University of Michigan,108000000,12536070,173,0.133195,7.0,8.0,7.0,5.0
6,7,University of Washington,127900000,5032077,133,0.092631,9.0,7.0,10.0,8.0
7,8,Cornell University,549000000,10471284,144,0.178513,5.0,3.0,8.0,7.0
8,9,Columbia University New York,166300000,13409770,120,0.122581,8.0,6.0,6.0,9.0
9,10,Johns Hopkins University,64500000,7279267,96,0.075731,10.0,10.0,9.0,10.0


In [ ]:
#Sort 'University Rank' in ascending order
df2_sorted = df2.sort_values(by=['University Rank'])
df2_sorted.to_csv('data_sorted.csv', index=False)
df2_sorted

,ID,University,Visibility Score,Openness Score,Excelence Score,Rank Score,University Rank,Visibility Rank,Openness Rank,Excelence Rank
0,1,Harvard University,3400000000,28141176,951,1.000000,1.0,1.0,1.0,1.0
2,3,Massachusetts Institute of Technology,279300000,15603655,845,0.451937,2.0,4.0,4.0,2.0
1,2,Stanford University,774500000,23006786,451,0.385347,3.0,2.0,2.0,3.0
3,4,University of California Berkeley,270200000,15700111,256,0.203202,4.0,5.0,3.0,4.0
7,8,Cornell University,549000000,10471284,144,0.178513,5.0,3.0,8.0,7.0
4,5,University of Oxford,101600000,14384636,165,0.135458,6.0,9.0,5.0,6.0
5,6,University of Michigan,108000000,12536070,173,0.133195,7.0,8.0,7.0,5.0
8,9,Columbia University New York,166300000,13409770,120,0.122581,8.0,6.0,6.0,9.0
6,7,University of Washington,127900000,5032077,133,0.092631,9.0,7.0,10.0,8.0
9,10,Johns Hopkins University,64500000,7279267,96,0.075731,10.0,10.0,9.0,10.0


In [ ]:
df2_result = pd.DataFrame(df2_sorted, columns=["University Rank"])
df2_result['University'] = df2_sorted['University']
df2_result['Visibility Rank'] = df2_sorted['Visibility Rank']
df2_result['Openness Rank'] = df2_sorted['Openness Rank']
df2_result['Excelence Rank'] = df2_sorted['University Rank']

df2_result

,University Rank,University,Visibility Rank,Openness Rank,Excelence Rank
0,1.0,Harvard University,1.0,1.0,1.0
2,2.0,Massachusetts Institute of Technology,4.0,4.0,2.0
1,3.0,Stanford University,2.0,2.0,3.0
3,4.0,University of California Berkeley,5.0,3.0,4.0
7,5.0,Cornell University,3.0,8.0,5.0
4,6.0,University of Oxford,9.0,5.0,6.0
5,7.0,University of Michigan,8.0,7.0,7.0
8,8.0,Columbia University New York,6.0,6.0,8.0
6,9.0,University of Washington,7.0,10.0,9.0
9,10.0,Johns Hopkins University,10.0,9.0,10.0


# 4. Compare the results with Webometrics

In [ ]:
#Webometrics ranking results (06/2023)
df3 = pd.read_csv("Webometrics Ranking.csv")
df3

,ID,University,Visibility Rank,Openness Rank,Excellence Rank,Webometrics Rank
0,1,Harvard University,1,1,1,1
1,2,Stanford University,3,2,2,2
2,3,Massachusetts Institute of Technology,2,3,7,3
3,4,University of California Berkeley,4,4,10,4
4,5,University of Oxford,9,5,3,5
5,6,University of Michigan,7,7,5,6
6,7,University of Washington,6,10,6,7
7,8,Cornell University,5,8,9,8
8,9,Columbia University New York,8,6,8,9
9,10,Johns Hopkins University,10,9,4,10


In [ ]:
#Compare ranking results
university_rank = df2['University Rank']
webometrics_rank = df3['Webometrics Rank']
print(university_rank)
print(webometrics_rank)

cosine_rank = np.dot(university_rank, webometrics_rank)/(norm(university_rank)*norm(webometrics_rank))

print("Cosine Similarity vs Webometrics:", cosine_rank)
cosine_rank

0     1.0
1     3.0
2     2.0
3     4.0
4     6.0
5     7.0
6     9.0
7     5.0
8     8.0
9    10.0
Name: University Rank, dtype: float64
0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: Webometrics Rank, dtype: int64
Cosine Similarity vs Webometrics: 0.9766233766233767


0.9766233766233767

# 5. Search for ranking

## 5.1. Search based on the input entered

In [ ]:
# Function to search for universities by name using a phrase and return a list of matching universities
def search_university(name):
    matched_universities = []
    for university in df2["University"]:
        if name.lower() in university.lower() or university.lower().find(name.lower()) != -1:
            matched_universities.append(university)
    if len(matched_universities) == 0:
        return "Không tìm thấy trường đại học nào phù hợp"
    else:
        return matched_universities

In [ ]:
name = input("Enter the search phrase: ")
search_result = search_university(name)
print(search_result)

Nhập cụm từ tìm kiếm: university
['Harvard University', 'Stanford University', 'University of California Berkeley', 'University of Oxford', 'University of Michigan', 'University of Washington', 'Cornell University', 'Columbia University New York', 'Johns Hopkins University']


In [ ]:
#Search results
search_df = pd.DataFrame(search_result, columns=["University"]);
search_df

,University
0,Harvard University
1,Stanford University
2,University of California Berkeley
3,University of Oxford
4,University of Michigan
5,University of Washington
6,Cornell University
7,Columbia University New York
8,Johns Hopkins University


In [ ]:
#Ranking results of the universities just searched
search_df = pd.DataFrame(search_result, columns=["University"]);

search_ranking = []
for i in range(len(search_result)):
  if search_result[i] in df2['University'].values:
    # Nếu tên trường đại học tồn tại, thực hiện truy xuất giá trị của nó
    search_ranking.append(df2['University Rank'][df2.loc[df2['University'] == search_result[i]].index[0]])
  else:
      # Nếu tên trường đại học không tồn tại trong DataFrame `df2`, thêm một giá trị mặc định hoặc thông báo lỗi tương ứng
      search_ranking.append(None)

search_df["Ranking"] = search_ranking
search_df

,University,Ranking
0,Harvard University,1.0
1,Stanford University,3.0
2,University of California Berkeley,4.0
3,University of Oxford,6.0
4,University of Michigan,7.0
5,University of Washington,9.0
6,Cornell University,5.0
7,Columbia University New York,8.0
8,Johns Hopkins University,10.0


## 5.2. Click on the name of one of the universities

In [ ]:
change_input = "University of Michigan"
df2_Visibility_Score = df2['Visibility Score'][df2.loc[df2['University'] == change_input].index[0]]
df2_Openness_Score = df2['Openness Score'][df2.loc[df2['University'] == change_input].index[0]]
df2_Excelence_Score = df2['Excelence Score'][df2.loc[df2['University'] == change_input].index[0]]
df2_Rank_Score = df2['Rank Score'][df2.loc[df2['University'] == change_input].index[0]]
df2_University_Rank = df2['University Rank'][df2.loc[df2['University'] == change_input].index[0]]
print("Visibility Score = ", df2_Visibility_Score)
print("Openness Score = " , df2_Openness_Score)
print("Excelence Score = " , df2_Excelence_Score)
print("Rank Score = " , df2_Rank_Score)
print("University Rank = " , df2_University_Rank)

Visibility Score =  108000000
Openness Score =  12536070
Excelence Score =  173
Rank Score =  0.13319493516388986
University Rank =  7.0


# 6. Try changing the criteria score and view the university ranking

In [ ]:
df2_sorted = df2.sort_values(by=['University Rank'])
df2_sorted.to_csv('data_sorted.csv', index=False)
df2_sorted

,ID,University,Visibility Score,Openness Score,Excelence Score,Rank Score,University Rank,Visibility Rank,Openness Rank,Excelence Rank
0,1,Harvard University,3400000000,28141176,951,1.000000,1.0,1.0,1.0,1.0
2,3,Massachusetts Institute of Technology,279300000,15603655,845,0.451937,2.0,4.0,4.0,2.0
1,2,Stanford University,774500000,23006786,451,0.385347,3.0,2.0,2.0,3.0
3,4,University of California Berkeley,270200000,15700111,256,0.203202,4.0,5.0,3.0,4.0
7,8,Cornell University,549000000,10471284,144,0.178513,5.0,3.0,8.0,7.0
4,5,University of Oxford,101600000,14384636,165,0.135458,6.0,9.0,5.0,6.0
5,6,University of Michigan,108000000,12536070,173,0.133195,7.0,8.0,7.0,5.0
8,9,Columbia University New York,166300000,13409770,120,0.122581,8.0,6.0,6.0,9.0
6,7,University of Washington,127900000,5032077,133,0.092631,9.0,7.0,10.0,8.0
9,10,Johns Hopkins University,64500000,7279267,96,0.075731,10.0,10.0,9.0,10.0


In [ ]:
#Example: 'University of Michigan'
change_input = "University of Michigan"
df2_sorted_Visibility_Score = df2_sorted['Visibility Score'][df2_sorted.loc[df2_sorted['University'] == change_input].index[0]]
df2_sorted_Openness_Score = df2_sorted['Openness Score'][df2_sorted.loc[df2_sorted['University'] == change_input].index[0]]
df2_sorted_Excelence_Score = df2_sorted['Excelence Score'][df2_sorted.loc[df2_sorted['University'] == change_input].index[0]]
df2_sorted_Rank_Score = df2_sorted['Rank Score'][df2_sorted.loc[df2_sorted['University'] == change_input].index[0]]
df2_sorted_University_Rank = df2_sorted['University Rank'][df2_sorted.loc[df2_sorted['University'] == change_input].index[0]]
print("Visibility Score = ", df2_sorted_Visibility_Score)
print("Openness Score = " , df2_sorted_Openness_Score)
print("Excelence Score = " , df2_sorted_Excelence_Score)
print("Rank Score = " , df2_sorted_Rank_Score)
print("University Rank = " , df2_sorted_University_Rank)

Visibility Score =  108000000
Openness Score =  12536070
Excelence Score =  173
Rank Score =  0.13319493516388986
University Rank =  7.0


In [ ]:
result = (float(df2_sorted['Visibility Score'][i] * 0.5))/max_value_VisibilityScore + (float(df2_sorted['Openness Score'][i] * 0.1))/max_value_OpennessScore + (float(df2_sorted['Excelence Score'][i] * 0.4))/max_value_ExcelenceScore
change_Visibility_Score = float(input("Enter VisibilityScore: "))
change_Openness_Score = float(input("Enter Openness Score: "))
change_Excelence_Score = float(input("Enter Excelence_Score: "))
change_df2_sorted_Rank_Score = (float(change_Visibility_Score * 0.5))/max_value_VisibilityScore + (change_Openness_Score * 0.1)/max_value_OpennessScore + (float(change_Excelence_Score * 0.4))/max_value_ExcelenceScore
change_df2_sorted_Rank_Score

Nhập VisibilityScore: 150000000
Nhập Openness Score: 12536070
Nhập Excelence_Score: 173


0.13937140575212514

In [ ]:
#sequence sorted in descending order
#find the first number greater than x
def binary_search_first_greater_than_x(arr, x):
    left = 0
    right = len(arr) - 1
    result = -1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] < x:
            right = mid - 1
        else:
            result = mid
            left = mid + 1
    return result

In [ ]:
change_rank_score = df2['Rank Score'].tolist()
change_rank_score.sort(reverse=True)
change_Ranking_Index = binary_search_first_greater_than_x(change_rank_score, change_df2_sorted_Rank_Score)
print("The university standing before: ",df2_sorted['University'][change_Ranking_Index])
print("Results after ranking:", change_Ranking_Index + 2)

Trường đại học đứng trước:  University of Oxford
Kết quả sau khi xếp hạng:  6
